In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
!wget https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip

--2019-06-02 08:24:14--  https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip
Resolving learner.csie.ntu.edu.tw (learner.csie.ntu.edu.tw)... 140.112.90.193
Connecting to learner.csie.ntu.edu.tw (learner.csie.ntu.edu.tw)|140.112.90.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3780741206 (3.5G) [application/zip]
Saving to: ‘ml19spring.zip’

ml19spring.zip      100%[===================>]   3.52G  16.3MB/s    in 5m 8s   

2019-06-02 08:29:24 (11.7 MB/s) - ‘ml19spring.zip’ saved [3780741206/3780741206]



In [0]:
!unzip -q ml19spring.zip
!ls

unzip:  cannot find or open ml19spring.zip, ml19spring.zip.zip or ml19spring.zip.ZIP.
sample_data


In [1]:
# Save model to your Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
DataDir='./gdrive/My Drive/term_2019_1/MLTech/Final/data/'
x_test = np.load(DataDir+'X_test.npz')['arr_0']
X = np.load(DataDir+'X_train.npz')['arr_0']
Y = np.load(DataDir+'Y_train.npz')['arr_0']

In [3]:
dirPath = './MLP_output/'
!mkdir ./MLP_output
print(X.shape)

(47500, 10000)


In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=False)
X = scaler.fit_transform(X)
x_test = scaler.transform( x_test )

In [0]:
important_fea = np.load(DataDir+'important_feat_0.0002.npy')
X=X[:,important_fea]
x_test = x_test[:,important_fea]

In [0]:
del x_train,x_val,y_train,y_val

In [5]:
from keras.models import Sequential
from keras.models import *
from keras.layers import *
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
x_train,x_val,y_train,y_val=train_test_split(X,Y,test_size=0.2,random_state=44)
print(x_train.shape,x_val.shape)

Using TensorFlow backend.


(38000, 10000) (9500, 10000)


In [0]:
del model
# 23.82688

In [0]:
del y_train_T,y_val_T

In [7]:
import pickle
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import Model
def MLP_MAE_onlyOutput(input_size):
    same_input = Input(shape=(input_size,), dtype='float32',name='input')
    X1 = Dense(512,kernel_initializer='normal')(same_input)
    X1 = PReLU()(X1)
    X1 = Dense(128,kernel_initializer='normal')(X1)
    X1 = PReLU()(X1)
    X1=BatchNormalization()(X1)
    X1 = Dense(68,kernel_initializer='normal',activation='tanh')(X1)
    X1=BatchNormalization()(X1)
    X1 = Dense(32,kernel_initializer='normal',activation='tanh')(X1)
    X1=BatchNormalization()(X1)
    y1 = Dense(1,kernel_initializer='normal',name='output_y1')(X1)
    
    X2 = Dense(512,kernel_initializer='normal')(same_input)
    X2 = PReLU()(X2)
    X2 = Dense(128,kernel_initializer='normal')(X2)
    X2 = PReLU()(X2)
    X2=BatchNormalization()(X2)
    X2 = Dense(68,kernel_initializer='normal',activation='tanh')(X2)
    X2=BatchNormalization()(X2)
    X2 = Dense(32,kernel_initializer='normal',activation='tanh')(X2)
#     X2=BatchNormalization()(X2)
    y2 = Dense(1,kernel_initializer='normal',name='output_y2')(X2)
    
    X3 = Dense(512,kernel_initializer='normal')(same_input)
    X3 = PReLU()(X3)
    X3 = Dense(64,kernel_initializer='normal')(X3)
    X3 = PReLU()(X3)
    y3 = Dense(1,kernel_initializer='normal',name='output_y3')(X3)
    model = Model(inputs=same_input, 
                 outputs=[y1,y2,y3])

    model.compile(optimizer='adam',
                  loss={'output_y1': 'mean_absolute_error',
                       'output_y2': 'mean_absolute_error',
                       'output_y3': 'mean_absolute_error'},loss_weights=[300,1,200])
    return model
def baseline_MLP(input_size):
    model = Sequential()
    model.add(Dense(512,input_dim=input_size,kernel_initializer='normal'))
    model.add(PReLU())
    model.add(Dense(128,kernel_initializer='normal'))
    model.add(PReLU())
    model.add(Dense(3,kernel_initializer='normal'))
    model.compile(loss = 'mean_absolute_percentage_error',optimizer='adam')
    return model
def baseline_MLP_WMAE(input_size):
    m_input = Input(shape=(input_size,), dtype='float32')
    X = Dense(512,kernel_initializer='normal')(m_input)
    X = PReLU()(X)
    X = Dense(128,kernel_initializer='normal')(X)
    X = PReLU()(X)
    X1 = Dense(68,kernel_initializer='normal',activation='tanh')(X)
    X1=BatchNormalization()(X1)
    X1 = Dense(32,kernel_initializer='normal',activation='tanh')(X)
    X1=BatchNormalization()(X1)
    #     X1 = PReLU()(X1)
    X2 = Dense(68,kernel_initializer='normal',activation='tanh')(X)
    X2=BatchNormalization()(X2)
    X2 = Dense(32,kernel_initializer='normal',activation='tanh')(X)
    X2=BatchNormalization()(X2)
    #     X2 = PReLU()(X2)
    X3 = Dense(68,kernel_initializer='normal',activation='tanh')(X)
    X3=BatchNormalization()(X3)
#     X3 = PReLU()(X3)
    y1 = Dense(1,kernel_initializer='normal')(X1)
    y2 = Dense(1,kernel_initializer='normal')(X2)
    y3 = Dense(1,kernel_initializer='normal')(X3)
    model = Model(inputs=m_input, outputs=[y1,y2,y3])
    model.compile(optimizer='adam', loss='mean_absolute_error',
              loss_weights=[300,1,200])
    return model
def cnn_WMAE(input1_size,input2_size):
    print(input1_size,input2_size)
    m_input1 = Input(shape=(5000,1,), dtype='float32')
    m_input2 = Input(shape=(100,50,), dtype='float32')
    x11 = Conv1D(64, 7,strides=4,input_shape=(5000,1), padding='same', activation='relu')(m_input1)
    x11 =MaxPooling1D(pool_size=3,strides=4)(x11)
    x11 = Conv1D(192, 3,strides=2, padding='same', activation='relu')(x11)
    x11 =MaxPooling1D(pool_size=3,strides=2)(x11)
    tower_1 = Conv1D(96, 3, padding='same', activation='relu')(x11)
    tower_1 = Conv1D(128, 3, padding='same', activation='relu')(tower_1)

    tower_2 = Conv1D(16, 1, padding='same', activation='relu')(x11)
    tower_2 = Conv1D(32, 5, padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling1D(3, strides=1, padding='same')(x11)
    tower_3 = Conv1D(32, 1, padding='same', activation='relu')(tower_3)
    tower_4 = Conv1D(16, 1, padding='same', activation='relu')(x11)

    x11 = concatenate([tower_1, tower_2, tower_3, tower_4], axis=-1)
    x11 = GlobalAveragePooling1D()(x11)
    
#     x11 = Flatten()(x11)
#     x12 = Flatten()(x12)
#     x2 = Conv2D(64, 5,strides=2,input_shape=(50,100,1), padding='same', activation='relu')(m_input2)
#     x2 =MaxPooling1D(pool_size=2,strides=2)(x2)
#     x2 = Conv2D(192, 3,strides=2, padding='same', activation='relu')(x2)
# #     x11 =MaxPooling1D(pool_size=3,strides=2)(x11)
#     tower_1 = Conv2D(96, 3, padding='same', activation='relu')(x2)
#     tower_1 = Conv2D(128, 3, padding='same', activation='relu')(tower_1)

#     tower_2 = Conv2D(16, 1, padding='same', activation='relu')(x2)
#     tower_2 = Conv2D(32, 5, padding='same', activation='relu')(tower_2)

#     tower_3 = MaxPooling1D(3, strides=1, padding='same')(x2)
#     tower_3 = Conv2D(32, 1, padding='same', activation='relu')(tower_3)
#     tower_4 = Conv2D(64, 1, padding='same', activation='relu')(x2)

#     x2 = concatenate([tower_1, tower_2, tower_3, tower_4], axis=-1)
#     x2 = GlobalAveragePooling2D()(x2)
    
    x21 = Conv1D(64,kernel_size=3,input_shape=(100,50,), activation='relu')(m_input2)
    x21 = Conv1D(32,kernel_size=1, activation='relu')(x21)
    x21 =MaxPooling1D(pool_size=24)(x21)

    x21 = Flatten()(x21)
    merged = concatenate(
        [x11, x21],
        axis=-1)
    merged = Dropout(0.15)(merged)
    merged=BatchNormalization()(merged)
    merged = Dense(units=200,kernel_initializer='normal')(merged)
    merged = PReLU()(merged)
    merged = Dense(units=64,kernel_initializer='normal', activation='tanh')(merged)
#     yy2 = Dense(units=64,kernel_initializer='normal')(merged)
#     yy2=PReLU()(yy2)
#     yy3 = Dense(units=64,kernel_initializer='normal', activation='tanh')(merged)
#     merged = Dropout(0.15)(merged)
#     merged = Dense(units=128,kernel_initializer='normal')(merged)
#     merged=PReLU()(merged)
#     merged = Dropout(0.2)(merged)
#     merged=BatchNormalization()(merged)
    y1 = Dense(units=1)(merged)
    y2 = Dense(units=1)(merged)
    y3 = Dense(units=1)(merged)

    model = Model(
        inputs=[m_input1, m_input2],
        outputs=[y1,y2,y3])
#     adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.98)
    model.compile(optimizer='adam', loss='mean_absolute_error',
              loss_weights=[300,1,200])
#     model.compile(optimizer='adam', loss='mean_absolute_error')
    return model
def resize_input(X):
    X1 = X[:,:5000].reshape(-1,5000,1)
    X2 = X[:,5000:].reshape(-1,50,100)
    X2 = np.transpose(X2,(0,2,1))
    print('X1 shape:',X1.shape,' X2 shape:',X2.shape)
    return X1,X2
dirPath = './MLP_output/' 
# model = baseline_MLP_WMAE(x_train.shape[1])
# model = baseline_MLP(x_train.shape[1])
# model = MLP_MAE_onlyOutput(x_train.shape[1])
x_train1,x_train2 = resize_input(x_train)
x_val1,x_val2 = resize_input(x_val)
model = cnn_WMAE(x_train1.shape[1:3],x_train2.shape[1:4])
model.summary()

BATCH_SIZE = 1024
NUM_EPOCHS = 200

# checkpoint
early_stopping = EarlyStopping(monitor='val_loss',patience=30)
MODELPATH=dirPath+"MLP_model.hdf5"
checkpoint = ModelCheckpoint(MODELPATH, monitor='val_loss', verbose=1, save_best_only=True)

print(y_train.shape)
y_train = np.transpose(y_train)
print(y_train.shape)
y_val = np.transpose(y_val)
print(y_val.shape)
# history = model.fit(x=x_train,y=[y_train_T[0],y_train_T[1],y_train_T[2]],batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
#         validation_data=(x_val,[y_val_T[0],y_val_T[1],y_val_T[2]]),shuffle=True,callbacks=[early_stopping,checkpoint])
# history = model.fit(x=x_train,y=y_train,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
#         validation_data=(x_val,y_val),shuffle=True,callbacks=[early_stopping,checkpoint])
# history = model.fit(x=x_train,y=y_train,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
#         validation_data=(x_val,y_val),shuffle=True,callbacks=[early_stopping,checkpoint])
history = model.fit(x=[x_train1,x_train2],y=[y_train[0],y_train[1],y_train[2]],batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,
        validation_data=([x_val1,x_val2],[y_val[0],y_val[1],y_val[2]]),shuffle=True,callbacks=[early_stopping,checkpoint])
file_his = open(dirPath+'MLP_history.pickle', 'wb')
pickle.dump(history.history, file_his)
file_his.close()
#25 100 114

W0615 11:48:08.715872 140417854199680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0615 11:48:08.725764 140417854199680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


X1 shape: (38000, 5000, 1)  X2 shape: (38000, 100, 50)
X1 shape: (9500, 5000, 1)  X2 shape: (9500, 100, 50)
(5000, 1) (100, 50)


W0615 11:48:08.844748 140417854199680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0615 11:48:08.907122 140417854199680 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 5000, 1)      0                                            
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 1250, 64)     512         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_5 (MaxPooling1D)  (None, 312, 64)      0           conv1d_11[0][0]                  
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 156, 192)     37056       max_pooling1d_5[0][0]            
__________________________________________________________________________________________________
max_poolin

In [8]:
import os
from keras.models import load_model
# MODELPATH1="CNN_11_model.hdf5"
if os.path.exists(MODELPATH):
#     model.load_weights(MODELPATH)
      model=load_model(MODELPATH)
    # 若成功加载前面保存的参数，输出下列信息
      print("checkpoint_loaded")
# Y_hat = model.predict(X)
# X1 = np.concatenate([x_train1,x_val1])
# X2 = np.concatenate([x_train2,x_val2])
X_1,X_2 = resize_input(X)
Y_hat = model.predict([X_1,X_2])
# Y_hat2 = model.predict([x_val1,x_val2])

checkpoint_loaded
X1 shape: (47500, 5000, 1)  X2 shape: (47500, 100, 50)


In [9]:
Y_hat=np.array(Y_hat)
Y_hat=Y_hat.reshape(3,-1)
print(Y_hat.shape)

(3, 47500)


In [0]:
# Y_hat=np.array(Y_hat)
# Y_hat = Y_hat.reshape(3,42750)
# Y_hat = np.transpose(Y_hat)
# print(Y_hat.shape)
# Y_hat2=np.array(Y_hat2)
# Y_hat2 = Y_hat2.reshape(3,4750)
# Y_hat2 = np.transpose(Y_hat2)
# print(Y_hat2.shape)
# Y_hat = np.concatenate([Y_hat,Y_hat2])
# print(Y_hat.shape)

ValueError: ignored

In [0]:
# yy=np.concatenate([y_train,y_val])
# print(yy.shape)

(47500, 3)


In [10]:
# Y_hat = model.predict(X)
# Y_hat=np.array(Y_hat)
# Y_hat = Y_hat.reshape(3,X1.shape[0])
# Y_hat=np.array(Y_hat)
Y_hat = np.transpose(Y_hat)
# Y_hat = np.transpose(Y_hat).reshape(-1,3)
print(Y_hat.shape)
err_mat = np.abs(Y-Y_hat)/Y
err_mat = np.sum(err_mat,axis=1)
err_mat = np.sum(err_mat)
print(err_mat/Y.shape[0])
#0.9704514192587647
#10000d 0.9945707841335775
#0.5936615853774215
#0.32381799619055995

(47500, 3)
0.44162815180735504


In [0]:
print(Y_hat.shape,Y.shape)

(47500, 3) (47500, 3)


In [11]:
err_WMAE = np.dot(np.abs(Y-Y_hat),np.array([300,1,200]))
err_WMAE = np.sum(err_WMAE)/Y.shape[0]
print(err_WMAE)
#145.68255323961225
#25.64659003243506

##49.38923480439816

43.63045013435174


In [0]:
model.save('MLP_WMAE_lastEpoch.h5')

In [12]:
Xtest_1,Xtest_2 = resize_input(x_test)
Y_test_hat = np.array(model.predict([Xtest_1,Xtest_2]))
Y_test_hat = Y_test_hat.reshape(3,-1)
Y_test_hat = np.transpose(Y_test_hat)

Y_test_hat[:,0]=np.clip(Y_test_hat[:,0],0,1)
Y_test_hat[:,1]=np.clip(Y_test_hat[:,1],25,250)
Y_test_hat[:,2]=np.clip(Y_test_hat[:,2],0.5,1)

np.savetxt(dirPath+'CNN_16.csv',Y_test_hat,delimiter=',')

X1 shape: (2500, 5000, 1)  X2 shape: (2500, 100, 50)


In [13]:
!zip cnn_output_16 MLP_output/*

  adding: MLP_output/CNN_16.csv (deflated 61%)
  adding: MLP_output/MLP_history.pickle (deflated 53%)
  adding: MLP_output/MLP_model.hdf5 (deflated 15%)


In [0]:
!mv cnn_output_16.zip '/content/gdrive/My Drive/term_2019_1/MLTech/Final'

In [0]:
!mv MLP_output/MLP_history.pickle MLP_output/MLP_history1.pickle

In [0]:
!cp '/content/gdrive/My Drive/term_2019_1/MLTech/Final/cnn_output_11.zip' ./


In [0]:
!unzip cnn_output_11.zip

Archive:  cnn_output_11.zip
replace MLP_output/CNN_11.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: CNN_11_1.csv
  inflating: CNN_11_1.csv            
replace MLP_output/MLP_history.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: CNN_11_hist.pickle
  inflating: CNN_11_hist.pickle      
replace MLP_output/MLP_model.hdf5? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: CNN_11_model.hdf5
  inflating: CNN_11_model.hdf5       


In [0]:
!rm MLP_output/CNN_11_2.csv